In [1]:
import re
import time

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml

import dill
import glob
import os

from datetime import datetime, timedelta

In [33]:
def get_thing(id, **args):
    '''A "thing" is BGG's designation for a physical item, 
       such as a board game, expansion, board game accessory, 
       etc.  The "id" supplied can have several numbers 
       separated by commas to retrieve more than one item 
       at a time.
    
       For more information see: https://boardgamegeek.com/wiki/page/BGG_XML_API2#
       
       **args can supply an arbitrary collection of options 
       (in the form of paramaters like key=value) that will 
       be appended into the query string, where these pairs 
       will be turned into strings like "key=value" and added 
       to the query string (preceded, of course, by an ampersand 
       to make it a separate element of the URL query string).  
       
       Returns:  A string for the "thing".  The only processing 
       done is to remove the newline and tab characters from 
       the string.  
    '''
    
    url = 'https://www.boardgamegeek.com/xmlapi2/thing?id=' + str(id).strip()
    for (k,v) in args.items():   #  Add the arbitrary (key,value) 
                                 #  pairs passed to the query string.
        url += '&' + str(k) + '=' + str(v)
        
    r = requests.get(url)
    if r.status_code == 404:
        return None
    while r.status_code == 202:
        time.sleep(5)
        r = requests.get(url)
    return re.sub('[\n\t]', '', r.text)

def add_options(url, own=None, preordered=None, prevowned=None, 
                fortrade=None, want=None, wanttobuy=None, 
                wanttoplay=None, wishlist=None, comment=None):
    '''A "utility" type of function to add elements to the query 
       string.  We assume that the parameters are {0,1} integer 
       values (if they are not "None").  Note that we will quietly 
       skip over these parameters if they are not 0 or 1, treating 
       them implicitly as "None" values.  
       
       Returns:  The url with the additional options added as 
       'key=value' parameters to the url.  
    '''
    if own in [0,1]:
        url += '&own=' + str(own)
    if prevowned in [0,1]:
        url += '&prevowned=' + str(prevowned)
    if preordered in [0,1]:
        url += '&preordered=' + str(preordered)
    if fortrade in [0,1]:
        url += '&fortrade=' + str(fortrade)
    if want in [0,1]:
        url += '&want=' + str(want)
    if wishlist in [0,1]:
        url += '&wishlist=' + str(wishlist)
    if wanttobuy in [0,1]:
        url += '&wanttobuy=' + str(wanttobuy)
    if wanttoplay in [0,1]:
        url += '&wanttoplay=' + str(wanttoplay)
    if comment in [0,1]:
        url += '&comment=' + str(comment)
    return url

def get_collection(bggUserName, own=None, preordered=None, 
                   prevowned=None, fortrade=None, want=None, 
                   wanttobuy=None, wanttoplay=None, 
                   wishlist=None, comment=None, 
                   save=True, cutoff=timedelta(hours=24)):
    '''For more information see:  https://boardgamegeek.com/wiki/page/BGG_XML_API2

       Get the board games, and then get the board game 
       expansions.  This is a quirk of the BGG xmlapi2 interface, 
       in that it will incorrectly return the expansions as 
       subtype="boardgame", so we make two calls to get the 
       boardgames, and then the expansions separately.
       
       Returns:  A pandas DataFrame with the designated boardgames 
       in the user's collection, with columns containing 
       information about the games such as the user rating, 
       number of plays, etc.  
       
       Note:  In an effort to reduce traffic, we will check
       if we have previously retrieved the collections within
       the previous 24 hour period.  If so, we just load and
       return that information, otherwise we will download the
       collection.  
    '''
    bggUserName = bggUserName.strip()
    
    #  Check:  Do we have a previous version of this
    #  collection that was retrieved in the last 24 hours? 
    #  If so, we use that.  Otherwise we get the collection
    #  information from BGG.
    files_to_check = glob.glob(f'collections/{bggUserName}-*.*')
    if files_to_check and (cutoff is not None):
        name = files_to_check[0]
        file_time_stamp = datetime(int(name[-18:-14]), int(name[-14:-12]), 
                            int(name[-12:-10]), int(name[-9:-7]), 
                            int(name[-7:-5]))
        now = datetime.now()
        if (now - file_time_stamp) <= cutoff:
            with open(name, 'rb') as f:
                glist = dill.load(f)
                return glist
    
    result = []
    for game_type in ['excludesubtype=boardgameexpansion', 
                      'subtype=boardgameexpansion']:
        url = f'https://www.boardgamegeek.com/xmlapi2/collection?username={bggUserName.strip()}' + \
              f'&{game_type}&stats=1'
        #  Add parameters to the url based on what was 
        #  passed to this function.
        url = add_options(url, own, preordered, prevowned, 
                          fortrade, want, wanttobuy, wanttoplay, 
                          wishlist, comment)
        r = requests.get(url)
        if r.status_code == 404:
            return None
        else:
            while r.status_code == 202:   ##  BGG says that 
                            ## it usually queues requests 
                            ## for a collection, so we 
                            ## must check for a 202 code, 
                            ## and sleep and try again if necessary.  
                time.sleep(8.5)
                r = requests.get(url)
            initial_res = re.sub('[\n\t]', '', r.text)
            #  Check if there was an error from BGG, such as 
            #  an invalid username.  Raise an exception if
            #  we find an error in the response.  
            error = BeautifulSoup(initial_res, 'lxml').find('error')
            if error:
                raise ValueError(f'{error.find("message").text}')
            result.extend(list(BeautifulSoup(initial_res, 'lxml').find_all('item')))
   
    glist = []
    for item in result:
        d = dict()
        d['id'] = int(item.attrs['objectid'])
        d['name'] = item.find('name').text
        d['subtype'] = item.attrs['subtype']
        if item.find('yearpublished'):
            d['yearpublished'] = int(item.find('yearpublished').text)

        d.update(item.find("status").attrs)
        d['numplays'] = int(item.find('numplays').text)
        d['lastmodified'] = pd.to_datetime(d['lastmodified'])
        if item.find('rating'):
            d['rating'] = item.find('rating').attrs['value']
            if d['rating'] == 'N/A':
                d['rating'] = np.nan
            else:
                d['rating'] = float(d['rating'])
        if item.find('comment'):
            d['comment'] = item.find('comment').text
        d['username'] = bggUserName
        glist.append(d)
    
    glist = pd.DataFrame(glist).set_index('id').sort_values('name')
    glist = glist[['name', 'subtype', 'yearpublished', 'own', 'prevowned', 'fortrade',
       'want', 'wanttoplay', 'wanttobuy', 'wishlist', 'preordered',
       'lastmodified', 'rating', 'numplays', 'wishlistpriority',
       'comment', 'username']]
    
    for column in ['yearpublished', 'own', 'prevowned', 
                   'fortrade', 'want', 'wanttoplay', 
                   'wanttobuy', 'wishlist', 'preordered', 
                   'numplays']:
        glist[column] = glist[column].fillna(-1).astype(np.int32)
    
    #  Do we save the collection information?  
    #  By default, we do, governed by the "save" parameter.
    if save:
        #  First remove any previous versions for this user
        files_to_delete = glob.glob(f'collections/{bggUserName}-*.dill')
        for f in files_to_delete:
            os.remove(f)
        
        now = datetime.strftime(datetime.now(), '%Y%m%d-%H%M')
        with open(f'collections/{bggUserName}-{now}.dill', 'wb') as f:
            dill.dump(glist, f)
        
    return glist

In [3]:
c = get_collection('craw-daddy', cutoff=None)

In [4]:
c

,name,subtype,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,rating,numplays,wishlistpriority,comment,username
id,,,,,,,,,,,,,,,,,
122711,"""La Garde recule!""",boardgame,2011,1,0,1,0,0,0,0,0,2015-01-15 11:02:06,NaN,0,NaN,NaN,craw-daddy
8257,&Cetera,boardgameexpansion,2013,1,0,0,0,0,0,0,0,2015-01-12 01:52:06,NaN,0,NaN,NaN,craw-daddy
153999,"...and then, we held hands.",boardgame,2015,1,0,0,0,0,0,0,0,2015-10-21 14:41:59,NaN,4,NaN,NaN,craw-daddy
27236,.45 Adventure: Crimefighting Action in the Pul...,boardgame,2006,1,0,0,0,0,0,0,0,2015-01-12 01:53:04,NaN,0,NaN,NaN,craw-daddy
155122,"1066, Tears to Many Mothers",boardgame,2018,1,0,0,0,0,0,0,0,2018-11-30 21:02:41,NaN,0,NaN,NaN,craw-daddy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248861,メトロックス (MetroX),boardgame,2018,1,0,0,0,0,0,0,0,2019-12-26 13:53:35,8.0,36,NaN,"One of the best ""roll and write"" games I've pl...",craw-daddy
278585,メトロックス (MetroX): Ishikawa Promo Map,boardgameexpansion,2018,1,0,0,0,0,0,0,0,2019-12-26 13:56:22,NaN,1,NaN,NaN,craw-daddy
181260,彩色島,boardgame,2015,0,1,0,0,0,0,0,0,2016-03-10 07:12:22,5.5,2,NaN,"Seems like some good ideas, but definitely not...",craw-daddy


In [5]:
len(c[c['own'] == 1])

1244

In [6]:
c['own']

id
122711    1
8257      1
153999    1
27236     1
155122    1
         ..
248861    1
278585    1
181260    0
158600    1
170198    1
Name: own, Length: 1839, dtype: int32

In [7]:
c[c['wishlist'] == 1]

,name,subtype,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,rating,numplays,wishlistpriority,comment,username
id,,,,,,,,,,,,,,,,,
3097,1849: The Game of Sicilian Railways,boardgame,1998,0,0,0,0,1,0,1,0,2019-10-20 22:34:45,NaN,0,3,NaN,craw-daddy
202617,18CLE,boardgame,2016,0,0,0,0,1,0,1,0,2019-01-01 00:04:48,NaN,0,2,NaN,craw-daddy
346248,18Korea,boardgame,2021,0,0,0,0,1,0,1,0,2021-11-08 20:02:32,NaN,0,2,NaN,craw-daddy
308305,21Moon,boardgame,2020,0,0,0,0,1,0,1,0,2021-10-21 12:30:29,NaN,0,2,NaN,craw-daddy
176588,A Glorious Chance: The Naval Struggle for Lake...,boardgame,2022,0,0,0,0,0,0,1,0,2021-11-08 20:00:13,NaN,0,4,NaN,craw-daddy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103185,Walnut Grove,boardgame,2011,0,0,0,1,0,0,1,0,2021-11-19 11:00:41,NaN,0,2,NaN,craw-daddy
67609,Way of the Dragon,boardgame,2012,0,0,0,0,0,0,1,0,2017-09-29 03:43:08,NaN,0,3,NaN,craw-daddy
347509,Wiñay Kawsay,boardgame,-1,0,0,0,0,1,0,1,0,2021-10-22 13:49:25,NaN,0,3,NaN,craw-daddy


In [8]:
len(c[c['wishlist'] == 1])

69

In [10]:
hopalong = get_collection('Hopalong', cutoff=None)

In [11]:
len(hopalong[hopalong['own'] == 1])

4288

In [12]:
helixx = get_collection('Helixx', cutoff=None)

In [13]:
helixx

,name,subtype,yearpublished,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,lastmodified,rating,numplays,wishlistpriority,comment,username
id,,,,,,,,,,,,,,,,,
153999,"...and then, we held hands.",boardgame,2015,1,0,0,0,0,0,0,0,2015-11-29 14:35:12,NaN,3,NaN,NaN,Helixx
63706,11 nimmt!,boardgame,2010,1,0,0,0,0,0,0,0,2011-10-28 18:30:35,NaN,0,NaN,NaN,Helixx
177590,13 Days: The Cuban Missile Crisis,boardgame,2016,0,0,0,0,0,0,0,0,2017-11-17 16:50:35,NaN,4,NaN,NaN,Helixx
203828,"13 Minutes: The Cuban Missile Crisis, 1962",boardgame,2017,0,0,0,0,0,0,0,0,2017-10-07 08:18:39,NaN,1,NaN,NaN,Helixx
193867,1822: The Railways of Great Britain,boardgame,2016,0,0,0,0,0,0,0,0,2020-10-25 19:05:05,NaN,0,NaN,NaN,Helixx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29387,Zombie Fluxx,boardgame,2007,1,0,0,0,0,0,0,0,2011-10-31 05:21:51,NaN,0,NaN,NaN,Helixx
117942,Zooloretto: The Dice Game,boardgame,2012,1,0,0,0,0,0,0,0,2019-02-11 08:13:00,NaN,0,NaN,NaN,Helixx
342210,echoes: The Cocktail,boardgame,2021,0,1,0,0,0,0,0,0,2022-01-01 10:30:32,NaN,1,NaN,NaN,Helixx


In [14]:
def getBGGCategories(save=False):
    '''Retrieve all of the boardgame categories used by BGG for classification.'''
    
    page = requests.get('https://boardgamegeek.com/browse/boardgamecategory')
    soup = BeautifulSoup(page.text)
    result = []
    for item in soup.findAll('td'):
        anchor = item.find('a')
        if anchor is not None:
            value = anchor.attrs['href'].split('/')[2]
            category = anchor.text
            result.append([value, category])
    result = pd.DataFrame(result, columns=['id','category']).set_index('id')
    
    if save:
        with open('data/boardGameCategories.dill', 'wb') as f:
            dill.dump(result, f)
            
    return result

In [15]:
boardGameCategories = getBGGCategories(save=True)

In [16]:
boardGameCategories

,category
id,
1009,Abstract Strategy
1032,Action / Dexterity
1022,Adventure
2726,Age of Reason
1048,American Civil War
...,...
1019,Wargame
1025,Word Game
1065,World War I


In [17]:
def getBGGMechanisms(save=False):
    '''Retrieve all of the boardgame mechanisms used by BGG for classification.'''
    
    mechs = []
    page = requests.get('https://boardgamegeek.com/browse/boardgamemechanic')
    soup = BeautifulSoup(re.sub('[\t\n]', '', page.text))
    for item in soup.findAll('td'):
        anchor = item.find('a')
        if anchor:
            c = anchor.attrs['href'].split('/')[2]
            m = anchor.text
            mechs.append((c,m))
    result = pd.DataFrame(mechs, columns=['id', 'mechanism']).set_index('id')
    
    if save:
        with open('data/boardGameMechanisms.dill', 'wb') as f:
            dill.dump(result, f)
            
    return result

In [18]:
boardGameMechanisms = getBGGMechanisms(save=True)

In [19]:
boardGameMechanisms

,mechanism
id,
2073,Acting
2838,Action Drafting
2001,Action Points
2689,Action Queue
2839,Action Retrieval
...,...
2017,Voting
2082,Worker Placement
2935,Worker Placement with Dice Workers


In [20]:
boardGameMechanisms[boardGameMechanisms['mechanism'].str.contains('Auction')]

,mechanism
id,
2012,Auction/Bidding
2930,Auction: Dexterity
2924,Auction: Dutch
2932,Auction: Dutch Priority
2918,Auction: English
2931,Auction: Fixed Placement
2923,Auction: Once Around
2920,Auction: Sealed Bid
2919,Auction: Turn Order Until Pass


In [21]:
def getGeekBuddies(bggUserName, save=True, cutoff=timedelta(days=7)):
    bggUserName = bggUserName.strip()
    
    #  Check if we have already gathered these geekbuddies.  We
    #  do so only every seven days, since I think that geekbuddies
    #  don't change all that often.
    files_to_check = glob.glob(f'geekbuddies/{bggUserName}-*.*')
    if files_to_check and (cutoff is not None):
        name = files_to_check[0]
        file_time_stamp = datetime(int(name[-18:-14]), int(name[-14:-12]), 
                            int(name[-12:-10]), int(name[-9:-7]), 
                            int(name[-7:-5]))
        now = datetime.now()
        if (now - file_time_stamp) <= cutoff:
            with open(name, 'rb') as f:
                buddies = dill.load(f)
                return buddies
    
    url = f'https://www.boardgamegeek.com/xmlapi2/user?name={bggUserName}&buddies=1'
    response = requests.get(url)
    response = BeautifulSoup(response.text, 'lxml')
    
    #  Check for a valid bggUserName
    name = response.find('firstname').attrs['value'] + response.find('lastname').attrs['value']
    if not name:
        raise ValueError(f'{error.find("message").text}')
    
    result = []
    for person in response.find_all('buddy'):
        result.append((int(person.attrs['id']), person.attrs['name'], bggUserName))
    buddies = pd.DataFrame(result,columns=['id', 'buddy', 'username']).set_index('id')

    #  Do we save the geekbuddies information?  
    #  By default, we do, governed by the "save" parameter
    if save:
        #  First remove any previous versions for this user
        files_to_delete = glob.glob(f'geekbuddies/{bggUserName}-*.dill')
        for f in files_to_delete:
            os.remove(f)
        
        now = datetime.strftime(datetime.now(), '%Y%m%d-%H%M')
        with open(f'geekbuddies/{bggUserName}-{now}.dill', 'wb') as f:
            dill.dump(buddies, f)

    return buddies

In [22]:
def getGame(bggGameId):
    response = BeautifulSoup(get_thing(bggGameId, stats=1))
    #print(response)
    if not response.find('item'):
        raise ValueError('Invalid BGG Game ID number')
        
    results = dict()
    results['id'] = int(response.find('item').attrs['id'])
    results['name'] = response.find('name').attrs['value']
    results['subtype'] = response.find('item').attrs['type']
    #  Clean the description up a little bit here
    results['description'] = response.find('description').text
    results['description'] = re.sub(r'&#10;|&mdash;|&ndash;', ' ', results['description'])
    results['description'] = re.sub(r'\s+', ' ', results['description'])
    
    results['minplayers'] = int(response.find('minplayers').attrs['value'])
    results['maxplayers'] = int(response.find('maxplayers').attrs['value'])
    results['averating'] = float(response.find('average').attrs['value'])
    results['bayesaverage'] = float(response.find('bayesaverage').attrs['value'])
    try:
        results['bggrank'] = int([r['value'] for r in response.find_all('rank') 
                                  if r.attrs['name'] == 'boardgame'][0])
    except ValueError:
        results['bggrank'] = -1
    results['averageweight'] = float(response.find('averageweight').attrs['value'])
    results['categories'] = [c['value'] for c in response.find_all('link') 
                             if c['type'] == 'boardgamecategory']
    results['mechanics'] = [c['value'] for c in response.find_all('link') 
                            if c['type'] == 'boardgamemechanic']
    results['family'] = [c['value'] for c in response.find_all('link') 
                         if c['type'] == 'boardgamefamily']
    results['designer'] = [c['value'] for c in response.find_all('link') 
                           if c['type'] == 'boardgamedesigner']
    results['artist'] = [c['value'] for c in response.find_all('link') 
                         if c['type'] == 'boardgameartist']
    results['publisher'] = [c['value'] for c in response.find_all('link') 
                            if c['type'] == 'boardgamepublisher']
    results['expansions'] = [int(c['id']) for c in response.find_all('link')
                             if c['type'] == 'boardgameexpansion']
    results['numratings'] = int(response.find('usersrated').attrs['value'])
    #print('-----------')
    #print(results)
    df = pd.DataFrame([results]).set_index('id')
    return df

In [23]:
Elfenland = getGame(10)

In [24]:
hopalong.loc[158]

name                                                        Elfengold
subtype                                            boardgameexpansion
yearpublished                                                    1999
own                                                                 1
prevowned                                                           0
fortrade                                                            0
want                                                                0
wanttoplay                                                          0
wanttobuy                                                           0
wishlist                                                            0
preordered                                                          0
lastmodified                                      2016-09-24 15:25:50
rating                                                            7.5
numplays                                                            0
wishlistpriority    

In [27]:
def getManyGames(desiredSet):
    result = []
    for i in desiredSet:
        try:
            game = getGame(i)
            result.append(game)
            time.sleep(1.5)
        except ValueError:
            print(f'Bad game number: {i}')
            
    return pd.concat(result).sort_values('id')

In [28]:
getManyGames(range(1,100))

Bad game number: 33
Bad game number: 35
Bad game number: 56
Bad game number: 77
Bad game number: 78
Bad game number: 81
Bad game number: 83
Bad game number: 84
Bad game number: 86
Bad game number: 87
Bad game number: 88
Bad game number: 89
Bad game number: 90
Bad game number: 92
Bad game number: 93
Bad game number: 94
Bad game number: 95
Bad game number: 96
Bad game number: 99


,name,subtype,description,minplayers,maxplayers,averating,bayesaverage,bggrank,averageweight,categories,mechanics,family,designer,artist,publisher,expansions,numratings
id,,,,,,,,,,,,,,,,,
1,Die Macher,boardgame,Die Macher is a game about seven sequential po...,3,5,7.61572,7.09696,327,4.3211,"[Economic, Negotiation, Political]","[Alliances, Area Majority / Influence, Auction...","[Country: Germany, Political: Elections, Serie...",[Karl-Heinz Schmiel],"[Marcus Gschwendtner, Harald Lieske]","[Hans im Glück, Moskito Spiele, Portal Games, ...",[],5424
2,Dragonmaster,boardgame,Dragonmaster is a trick-taking card game based...,3,4,6.64431,5.77688,4124,1.9630,"[Card Game, Fantasy]",[Trick-taking],"[Components: Gems/Crystals, Creatures: Dragons]","[G. W. ""Jerry"" D'Arcey]",[Bob Pepper],"[E.S. Lowe, Milton Bradley]",[],562
3,Samurai,boardgame,Samurai is set in medieval Japan. Players comp...,2,4,7.45685,7.23695,235,2.4849,"[Abstract Strategy, Medieval]","[Area Majority / Influence, Hand Management, H...",[Components: Map (Continental / National scale...,[Reiner Knizia],[Franz Vohwinkel],"[Fantasy Flight Games, Hans im Glück, 999 Game...",[],15319
4,Tal der Könige,boardgame,When you see the triangular box and the luxuri...,2,4,6.60989,5.67930,5417,2.6667,[Ancient],"[Action Points, Area Majority / Influence, Auc...","[Containers: Triangular Boxes, Country: Egypt,...",[Christian Beierer],[Thomas di Paolo],[KOSMOS],[],346
5,Acquire,boardgame,"In Acquire, each player strategically invests ...",2,6,7.33792,7.13744,300,2.5012,"[Economic, Territory Building]","[Hand Management, Investment, Market, Square G...",[Series: 3M Bookshelf Series],[Sid Sackson],"[Scott Okumura, Peter Whitley]","[3M, The Avalon Hill Game Co, Avalon Hill Game...","[324384, 323173]",18852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Bobby Lee: The Civil War in Virginia 1861-1865,boardgame,Bobby Lee covers the eastern theater of the Am...,2,2,6.72776,5.69948,5095,2.9184,"[American Civil War, Wargame]","[Dice Rolling, Secret Unit Deployment]","[Components: Block Wargames, Country: USA, Cro...",[Tom Dalgliesh],[Eric Hotz],[Columbia Games],[],356
85,Quebec 1759,boardgame,"Quebec 1759 is a small, simple and short war g...",2,2,6.96408,5.90131,3127,2.1512,"[Age of Reason, Wargame]","[Dice Rolling, Point to Point Movement, Secret...","[Cities: Quebec (Canada), Components: Block Wa...","[Steve Brewster, Tom Dalgliesh, Lance Gutteridge]",[],"[Columbia Games, Gamma Two Games]",[],713
91,Paths of Glory,boardgame,(from GMT Games' website:) They called it the ...,2,2,8.05135,7.32344,185,3.8402,"[Wargame, World War I]","[Campaign / Battle Card Driven, Dice Rolling, ...","[Continents: Europe, Players: Two Player Only ...",[Ted Raicer],"[Charles Kibler, Terry Leeds, Rodger B. MacGow...","[GMT Games, Devir, DiceTree Games, Udo Grebe G...","[329723, 26916]",4746


In [32]:
getGame(91)

,name,subtype,description,minplayers,maxplayers,averating,bayesaverage,bggrank,averageweight,categories,mechanics,family,designer,artist,publisher,expansions,numratings
id,,,,,,,,,,,,,,,,,
91,Paths of Glory,boardgame,(from GMT Games' website:) They called it the ...,2,2,8.05135,7.32344,185,3.8402,"[Wargame, World War I]","[Campaign / Battle Card Driven, Dice Rolling, ...","[Continents: Europe, Players: Two Player Only ...",[Ted Raicer],"[Charles Kibler, Terry Leeds, Rodger B. MacGow...","[GMT Games, Devir, DiceTree Games, Udo Grebe G...","[329723, 26916]",4746
